In [1]:
import sys
print(sys.version)

3.6.8 |Anaconda, Inc.| (default, Dec 30 2018, 01:22:34) 
[GCC 7.3.0]


In [56]:
import gpflow
import numpy as np
import math
import tensorflow as tf


In [57]:
import get_history as gh

In [58]:
import feather
import pandas as pd

In [59]:
def gather_cols(params, indices, name=None):
    """Gather columns of a 2D tensor.

    Args:
        params: A 2D tensor.
        indices: A 1D tensor. Must be one of the following types: ``int32``, ``int64``.
        name: A name for the operation (optional).

    Returns:
        A 2D Tensor. Has the same type as ``params``.
    """
    with tf.op_scope([params, indices], name, "gather_cols") as scope:
        # Check input
        params = tf.convert_to_tensor(params, name="params")
        indices = tf.convert_to_tensor(indices, name="indices")
        try:
            params.get_shape().assert_has_rank(2)
        except ValueError:
            raise ValueError('\'params\' must be 2D.')
        try:
            indices.get_shape().assert_has_rank(1)
        except ValueError:
            raise ValueError('\'params\' must be 1D.')

        # Define op
        p_shape = tf.shape(params)
        p_flat = tf.reshape(params, [-1])
        i_flat = tf.reshape(tf.reshape(tf.range(0, p_shape[0]) * p_shape[1],
                                       [-1, 1]) + indices, [-1])
        return tf.reshape(tf.gather(p_flat, i_flat),
                          [p_shape[0], -1])

In [67]:
np.eye(6)+.1*np.ones((6,6))

array([[1.1, 0.1, 0.1, 0.1, 0.1, 0.1],
       [0.1, 1.1, 0.1, 0.1, 0.1, 0.1],
       [0.1, 0.1, 1.1, 0.1, 0.1, 0.1],
       [0.1, 0.1, 0.1, 1.1, 0.1, 0.1],
       [0.1, 0.1, 0.1, 0.1, 1.1, 0.1],
       [0.1, 0.1, 0.1, 0.1, 0.1, 1.1]])

In [54]:
gpflow.Param([[1.0,0],[1,0]], transform=gpflow.transforms.positive,dtype=gpflow.settings.float_type)

,class,prior,transform,trainable,shape,fixed_shape,value
Parameter,Parameter,None,+ve,True,"(2, 2)",True,"[[1.0, 0.0], [1.0, 0.0]]"


In [86]:
gr = 10*np.eye(100)
gr.shape

(100, 100)

In [117]:



class CustomKernel(gpflow.kernels.Kernel):
    def __init__(self,input_dim, mysession=None,rhos=None,select_users=None,
                 active_dims=None, ARD=None, name=None):
        super().__init__(input_dim)

        self.sigma_u = gpflow.Param([[1.0,0.1],[0.1,1.0]], transform=gpflow.transforms.positive,
                                    dtype=gpflow.settings.float_type)
        
        #tf.constant(np.array([[1.0,0.1],[0.1,1.0]]))
        
        self.sigma_theta = gpflow.Param(np.array([[1.1, 0.1, 0.1, 0.1, 0.1, 0.1],
       [0.1, 1.1, 0.1, 0.1, 0.1, 0.1],
       [0.1, 0.1, 1.1, 0.1, 0.1, 0.1],
       [0.1, 0.1, 0.1, 1.1, 0.1, 0.1],
       [0.1, 0.1, 0.1, 0.1, 1.1, 0.1],
       [0.1, 0.1, 0.1, 0.1, 0.1, 1.1]]), transform=gpflow.transforms.positive,
                                    dtype=gpflow.settings.float_type)
        #print(self.sigma_theta)
        #gpflow.Param(1.0, transform=gpflow.transforms.positive,
                                  #dtype=gpflow.settings.float_type)
        
        self.sigma_v =  gpflow.Param(np.array([[10.0,0.0],[0.0,10.0]]), transform=gpflow.transforms.positive,
                                    dtype=gpflow.settings.float_type)
        #gpflow.Param(1.0, transform=gpflow.transforms.positive,
                                  #dtype=gpflow.settings.float_type)
        self.mysession=mysession
        
        self.rhos = tf.constant(rhos)
        self.select_users = tf.constant(select_users)
        ##will it freak out if there is no parameter?
        
    
    @gpflow.params_as_tensors
    def rbf_custom(self, X, X2=None):
        #print(X)
        #print(X2)
        if X2 is None:
             X2=X
        return tf.exp(-tf.divide(tf.square(tf.subtract(X,X2)),tf.constant(1.0,dtype=tf.float64)))
        return tf.constant(1.0,dtype=tf.float64)
    #tf.exp(-tf.subtract(X,X2) / float(2.2))
    
    @gpflow.params_as_tensors
    def get_all_sigmas(self,sigma_v,g_one,g_two,day_one,day_two):
    #sigmax = [[] for i in range(100)]
    #tf.placeholder(tf.float64, shape=(100, 2,2))
    
            
        #print('eff')
        #print(effects.shape)
        #user_term = tf.constant(np.array([[0.0,0.0],[0.0,0.0]]))           
        


        #print('eff')
        #print(effects.shape)
        #effects = tf.tensordot(effects,tf.transpose(g_two),axes = [[1],[0]])
    
    
    
        
        
        get_g = g_one
        get_g_two = g_two
        to_return = [[] for i in range(100)]
        for i in range(100):
            temp = [[] for i in range(100)]
            for j in range( 100):
                rho = self.rbf_custom(day_one[i], X2=day_two[j])
            #print(sess.run(rho))
                val = rho[0]
        #tf.scalar_mul(val,sigma_v)
            #print(val)
                this_sigma = tf.scalar_mul(val,sigma_v)
            #print(this_sigma)
                effects = tf.tensordot(get_g[i],this_sigma,axes=[[0],[1]])
                effects = tf.tensordot(effects,get_g_two[j],axes = [[0],[0]])
                temp[j]=effects
            to_return[i]=temp
        
        return to_return
    
    @gpflow.params_as_tensors
    def K(self, X, X2=None):
 

      
        
        f_one = gather_cols(X, [0,1,2,3,4,5], name=None)
        g_one = gather_cols(X, [6,7], name=None)
        user_id_one = gather_cols(X, [8], name=None)
        day_one = gather_cols(X, [9], name=None)



        #print(f_one.get_shape())
        
        if not X2 is None:
            print('called')
            f_two = gather_cols(X2, [0,1,2,3,4,5], name=None)
            g_two = gather_cols(X2, [6,7], name=None)  
            user_id_two = gather_cols(X2, [8], name=None)
            day_two = gather_cols(X2, [9], name=None)
            
         
            
        else:
            #print('called')
            user_id_two = X[-2]
            day_two = day_one
            f_two = f_one
            g_two=g_one
            
            
        
        
        #rho_term = self.rbf_custom(day_one,X2=day_two)
        #print('rho')
        #print(rho_term.get_shape())
        
        baselines = tf.tensordot(tf.transpose(f_one),self.sigma_theta,axes=[[0],[1]])
        #print('baselines')
        #print(baselines.get_shape())
        baselines = tf.tensordot(baselines,tf.transpose(f_two),axes=[[1],[0]])
        
        
        #rhos = tf.constant(np.array([[self.rbf_custom(day_one[i], X2=day_two[j])[0]] \
                                     #for i, j in zip([i for i in range(100)], [j for j in range(100)])]))
        
        
        #effects = self.get_all_sigmas(self.sigma_v,g_one,g_two,day_one,day_two)
            #print('look at shape')t
            #print(baselines)
        #print('baselines')
        #print(baselines.get_shape())
        
        #effect_term = tf.tensordot(rho_term,self.sigma_v,axes=[[0],[0]])

        #user_term = tf.constant(np.array([[0.0,0.0],[0.0,0.0]]))           
        #effect_term = tf.add(user_term,effect_term)
       #print('eff')
        #print(effect_term.shape)
        
        #print('eff')
        #print(effects.shape)
        #user_term = tf.constant(np.array([[0.0,0.0],[0.0,0.0]]))           
        


        #print('eff')
        #print(effects.shape)
        
        ##important
        effects = tf.tensordot(tf.transpose(g_one),self.sigma_u,axes=[[0],[1]])
       
        
        effects = tf.tensordot(effects,tf.transpose(g_two),axes = [[1],[0]])
        effects_one = tf.multiply(effects,self.select_users)
        
        
        
        effects = tf.tensordot(tf.transpose(g_one),self.sigma_v,axes=[[0],[1]])
       
        
        effects = tf.tensordot(effects,tf.transpose(g_two),axes = [[1],[0]])
        effects_two = tf.multiply(effects,self.rhos)
        
        effects = tf.add(effects_one,effects_two)
   
        
        #print('eff')
        #print(effects.shape)
        result = tf.add(baselines,effects)
        
        noise = 1000*np.eye(100)
        result = tf.add(result,tf.constant(noise))
        #effect_term = tf.add(user_term,effect_term)
        print('r')
        print(result.shape)
        
        
        return result
       
        
        #self.variance * tf.minimum(X, tf.transpose(X2))


In [61]:
##need to reformat history
def put_x_together(h):
    X = []
    
    for i in range(len(h.states)):
        state_vector = h.states[i]
        state_vector.append(float(h.users[i]))
        state_vector.append(float(h.day_function[i]))
        X.append(state_vector)
    return X


In [62]:
history = gh.get_history(8,100,5)


In [63]:
X = put_x_together(history)


In [64]:
X = np.array(X)


In [1]:
users


NameError: name 'users' is not defined

In [65]:
users = np.array([[float(X[i][8]==X[j][8]) for j in range(100)] for i in range(100)])


In [66]:
def rbf_custom_np( X, X2=None):
        #print(X)
        #print(X2)
        if X2 is None:
             X2=X
        return math.exp(-((X-X2)**2)/1.0)
        #tf.divide(tf.square(-tf.subtract(X,X2)),tf.constant(1.0,dtype=tf.float64)))
rdayone = [x[9] for x in X]
rdaytwo = rdayone
rhos = np.array([[rbf_custom_np( rdayone[i], X2=rdaytwo[j]) for j in range(100)] for i in range(100)])



In [67]:
y = np.array([[float(r)] for r in history.rewards])


In [118]:
sess = tf.Session()
k =  CustomKernel(7,mysession=sess,rhos=rhos,select_users=users)
m = gpflow.models.GPR(X,y, kern=k)


r
(100, 100)


In [119]:
gpflow.train.ScipyOptimizer().minimize(m,session=sess)


INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: 542.208214
  Number of iterations: 29
  Number of functions evaluations: 33


INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: 542.208214
  Number of iterations: 29
  Number of functions evaluations: 33


In [120]:

term= m.kern.K(X,X2=X)


called


r
(100, 100)


In [121]:
trm = term.eval(session=sess)


In [125]:
learned_s_theta = m.kern.sigma_theta.value

In [123]:
m.as_pandas_table()


,class,prior,transform,trainable,shape,fixed_shape,value
GPR/kern/sigma_theta,Parameter,None,+ve,True,"(6, 6)",True,"[[1e-06, 1e-06, 1e-06, 1e-06, 1e-06, 1e-06], [..."
GPR/kern/sigma_u,Parameter,None,+ve,True,"(2, 2)",True,"[[1e-06, 1e-06], [1e-06, 1e-06]]"
GPR/kern/sigma_v,Parameter,None,+ve,True,"(2, 2)",True,"[[1e-06, 1.0000000002220446e-06], [1.000000000..."
GPR/likelihood/variance,Parameter,None,+ve,True,(),True,1803.5640939878183


In [369]:
def obtain_kernel_and_sigma_u(all_xs,all_ys):
    k =  CustomKernel(all_xs.shape[1])
    m = gpflow.models.GPR(all_xs, all_ys, kern=k)
    gpflow.train.ScipyOptimizer().minimize(m,session=sess)
    #sigma_u = m.kern.sigma_u
    #.value
    cov = np.zeros((all_xs.shape[0],all_xs.shape[0]))
    term = m.kern.K(all_xs,X2=all_xs)
    print(m.as_pandas_table())
    #for i in range(all_xs.shape[0]):
        #for j in range(all_xs.shape[0]):
            #print(type(all_xs[i][0]))
           # term = m.kern.K(all_xs[i],all_xs[j])
            #term = tf.cast(term, tf.float32)
           # cov[i][j]=term
            #term.eval(session=sess)
            #term.eval(session=sess)
    return term.eval(session=sess)
#,sigma_u

In [370]:
cov = obtain_kernel_and_sigma_u(X,y)


(?, ?)
Tensor("GPR-e94f3f47-854/likelihood_1/strided_slice_1:0", dtype=float64)
Tensor("GPR-e94f3f47-854/likelihood_1/strided_slice_4:0", dtype=float64)
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: 542.233190
  Number of iterations: 32
  Number of functions evaluations: 33


INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: 542.233190
  Number of iterations: 32
  Number of functions evaluations: 33


(100, 3)
called


[17.62061977 16.3121152  16.83757731 15.21358701 18.68006498  3.
 15.        ]
[17.62061977 16.3121152  16.83757731 15.21358701 18.68006498  3.
 15.        ]
                             class prior transform  trainable shape  \
GPR/likelihood/variance  Parameter  None       +ve       True    ()   

                         fixed_shape               value  
GPR/likelihood/variance         True  2684.0070684618827  


In [128]:
def get_RT(y,X,sigma_theta):
    
    to_return = [y[i]-np.dot(X[i][0:6],np.ones(6)) for i in range(len(X))]
    return to_return

In [129]:
r_adjusted = get_RT(y,X,sigma_theta)

In [130]:
def create_H():
    return np.transpose(np.array([[1,0,0,0,0,0],[0,0,1,0,1,0]]))
    
    

In [131]:
def get_MT(new_x,sigma_theta,sigma_u,sigma_v,history,H):
    
    psi = new_x[6:8]
    user_id = new_x[8]
    day_id = new_x[9]
    
    to_return = [[] for i in range(history.shape[0])]
    
    for x_old_i in range(len(history)):
        x_old = history[x_old_i]
        phi = x_old[:6]
       
        old_user_id = x_old[8]
        old_day_id = x_old[9]
        
        inner = float(old_user_id==user_id)*sigma_u+rbf_custom_np(day_id,old_day_id)*sigma_v
        #print(inner.shape)
        #print(H.shape)
        inner = np.dot(H,inner)
        #print(inner.shape)
        inner = np.dot(inner,np.transpose(H))
        #print(inner.shape)
        inner = np.add(sigma_theta,inner)
        
        
        
        
        term = np.dot(np.transpose(phi),inner)
        to_return[x_old_i]=[i for i in term]
    return to_return


In [18]:
    sigma_theta = np.array([[1.1, 0.1, 0.1, 0.1, 0.1, 0.1],
       [0.1, 1.1, 0.1, 0.1, 0.1, 0.1],
       [0.1, 0.1, 1.1, 0.1, 0.1, 0.1],
       [0.1, 0.1, 0.1, 1.1, 0.1, 0.1],
       [0.1, 0.1, 0.1, 0.1, 1.1, 0.1],
       [0.1, 0.1, 0.1, 0.1, 0.1, 1.1]])
    sigma_u = np.array([[1.0,0.1],[0.1,1.0]])
    
    sigma_v = np.array([[10.0,0.0],[0.0,10.0]])
    
    

In [19]:
with open('testX.pkl','rb') as f:
    t = pickle.load(f)

In [48]:
#tst =get_MT(t[0],sigma_theta,sigma_u,sigma_v,t,create_H())

(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)
(6, 6)
(6, 2)

In [133]:
all_MS = [get_MT(x,sigma_theta,sigma_u,sigma_v,X,create_H()) for x in X]


In [135]:
#all_MS[0]

In [118]:
#one = tf.constant([0,0])
#two = tf.constant([1,1])
inp= tf.constant([[1.0, 2, 3], [4, 5, 6.0], [7.0, 8, 9]])
tf.norm([inp,inp],ord=2)


<tf.Tensor 'norm_9/Squeeze:0' shape=() dtype=float32>

In [126]:
df = pd.DataFrame(X,columns=['f_1','f_2','f_3','f_4','f_5','f_6','g_1','g_2','user_id','user_day_id'])
path = 'X_data.feather'
feather.write_dataframe(df, path)
df_rewards = pd.DataFrame(y,columns=['reward'])
path = 'y_data.feather'
feather.write_dataframe(df_rewards, path)

In [113]:
#path = 'adjusted_R.feather'
#feather.write_dataframe(df, path)
df_rewardsa = pd.DataFrame(r_adjusted,columns=['reward_centered'])
path = 'adjusted_R.feather'
feather.write_dataframe(df_rewardsa , path)

In [136]:
trm

array([[31151.07511374, 33229.2577411 , 31246.14996826, ...,
        29977.84463435, 29732.87325126, 29964.07900045],
       [33229.2577411 , 37610.47091257, 34427.60527294, ...,
        33039.88677868, 32762.35846839, 33014.83012101],
       [31246.14996826, 34427.60527294, 33373.76040804, ...,
        31066.54338706, 30805.94330263, 31044.38989066],
       ...,
       [29977.84463435, 33039.88677868, 31066.54338706, ...,
        30794.9554853 , 29551.69047504, 29791.74692022],
       [29732.87325126, 32762.35846839, 30805.94330263, ...,
        29551.69047504, 30303.45553549, 29541.5013214 ],
       [29964.07900045, 33014.83012101, 31044.38989066, ...,
        29791.74692022, 29541.5013214 , 30765.70883504]])

In [137]:
import pandas

In [141]:
from rpy2.robjects import numpy2ri
import rpy2.robjects as robjects
write_csv = robjects.r('write.csv')
write_csv(all_MS,'filename.csv')
#r("save(foo, file='here.gzip', compress=TRUE)"

rpy2.rinterface.NULL

In [132]:
import pickle
with open('temp_M.pkl','wb') as f:
    pickle.dump(all_MS,f)

In [137]:
df_rewardsa = pd.DataFrame(trm,columns=['u_{}'.format(i) for i in range(100)])
path = 'covar_from_kernel_on_train.feather'
feather.write_dataframe(df_rewardsa , path)


In [138]:
learned_s_theta = m.kern.sigma_theta.value
df_rewardsa = pd.DataFrame(learned_s_theta,columns=['i_{}'.format(i) for i in range(6)])
path = 'sigma_theta.feather'
feather.write_dataframe(df_rewardsa , path)


In [139]:
learned_s_u = m.kern.sigma_u.value
df_rewardsa = pd.DataFrame(learned_s_u,columns=['i_{}'.format(i) for i in range(2)])
path = 'sigma_u.feather'
feather.write_dataframe(df_rewardsa , path)

In [140]:
learned_s_v = m.kern.sigma_v.value
df_rewardsa = pd.DataFrame(learned_s_v,columns=['i_{}'.format(i) for i in range(2)])
path = 'sigma_v.feather'
feather.write_dataframe(df_rewardsa , path)


In [125]:
pp

[[1, 1, 2, 2, 2, 3, 3, 3], [1, 1, 2, 2, 2, 3, 3, 3]]

In [32]:
    def rbf_custom( X, X2=None):
        #print(X)
        #print(X2)
        if X2 is None:
             X2=X
        return tf.exp(tf.divide(tf.square(-tf.subtract(X,X2)),tf.constant(1.0,dtype=tf.float64)))
        return tf.constant(1.0,dtype=tf.float64)

In [519]:
tx = tf.constant(X)
day_two = gather_cols(tx, [6], name=None)
z= rbf_custom(day_two,day_two)

In [520]:
z

<tf.Tensor 'Exp_756:0' shape=(100, 1) dtype=float64>

In [521]:
def get_all_sigmas(sigma_v,g_one,g_two,day_one,day_two):
    #sigmax = [[] for i in range(100)]
    #tf.placeholder(tf.float64, shape=(100, 2,2))
    
            
        #print('eff')
        #print(effects.shape)
        #user_term = tf.constant(np.array([[0.0,0.0],[0.0,0.0]]))           
        


        #print('eff')
        #print(effects.shape)
        #effects = tf.tensordot(effects,tf.transpose(g_two),axes = [[1],[0]])
    
    get_g = g_one
    get_g_two = g_two
    to_return = [[] for i in range(get_g.get_shape()[0])]
    for i in range(get_g.get_shape()[0]):
        temp = [[] for i in range(get_g_two.get_shape()[0])]
        for j in range( get_g_two.get_shape()[0]):
            rho = rbf_custom(day_one[i], X2=day_two[j])
            #print(sess.run(rho))
            val = sess.run(rho)[0]
        #tf.scalar_mul(val,sigma_v)
            #print(val)
            this_sigma = tf.scalar_mul(val,sigma_v)
            #print(this_sigma)
            effects = tf.tensordot(get_g[i],this_sigma,axes=[[0],[1]])
            effects = tf.tensordot(effects,get_g_two[j],axes = [[0],[0]])
            temp[j]=effects
        to_return[i]=temp
        
    return to_return

In [465]:
(sess.run(z[0])[0])

1.0

In [28]:
        day_one = gather_cols(X, [6], name=None)
        f_one = gather_cols(X, [0,1,2], name=None)
        g_one = gather_cols(X, [3,4], name=None)

In [38]:
rbf_custom(day_one[0], X2=day_two[0])[0]

<tf.Tensor 'strided_slice_100004:0' shape=() dtype=float64>

In [30]:
g_two=g_one
day_two=day_one

In [49]:
ts = tf.stack([day_one,day_two],axis=1)

In [54]:
elems = ts

alternate = tf.map_fn(lambda x: rbf_custom(x[0], X2=x[1]), elems, dtype=(tf.tensor))

AttributeError: module 'tensorflow' has no attribute 'tensor'

In [72]:
rhos.shape

(100, 100)

In [433]:
sigma_theta =tf.constant(np.array([[100.0,10.0,10.0],[10.0,100.0,10.0],\
                                    [10.0,10.0,100.0]]))
sigma_v =  tf.constant(np.array([[100.0,0.0],[0.0,100.0]]))
sigma_u =  tf.constant(np.array([[100.0,10.0],[10.0,100.0]]))

In [199]:
user_id_one = X_one[-2]
day_one = X_one[-1]
f_one = X_one[0:3]
g_one = X_one[3:5]

user_id_two = X_two[-2]
day_two = X_two[-1]
f_two = X_two[0:3]
g_two = X_two[3:5]
        
        
rho_term = rbf_custom(day_one,X2=day_two).eval(session=sess) 
#print(sess.run(rho_term))
print('rho')
#print(tf.shape(f_one))
print(rho_term)
print(f_one.shape)
print(f_one)
#print(rho_term.get_shape()[1])
            #rho_term = tf.constant(np.array([1.0]))
       
baselines = tf.tensordot(tf.transpose(f_one),sigma_theta,axes=[[0],[0]])
print('baselines')
print(baselines.get_shape())
baselines = tf.tensordot(tf.transpose(baselines),f_two,axes=[[0],[0]])
            #print('look at shape')
            #print(baselines)
print('baselines')
print(baselines.get_shape())
print(baselines.eval(session=sess))
#if user_id_one==user_id_two:
#    user_term = self.sigma_u
#else:
#print('user')
#print('user_term')

            #print(self.sigma_v.shape)
            #print(rho_term.shape)
effect_term = tf.scalar_mul(rho_term,sigma_v)
#print('effect term')
#print(effect_term.shape)
#print('user term')
#print(user_term.shape)
user_term = tf.constant(np.array([[0.0,0.0],[0.0,0.0]]))           
effect_term = tf.add(user_term,effect_term)
#print('eff')
#print(effect_term.shape)
#print('g')
#print(g_one.shape)
effects = tf.tensordot(tf.transpose(g_one),effect_term,axes=[[0],[1]])
print('eff')
print(effects.shape)
effects = tf.tensordot(tf.transpose(effects),g_two,axes = [[0],[0]])
print('eff')
print(effects.shape)
result = tf.add(baselines,effects)
print('res')
print(result.shape)
print(result.eval(session=sess))

rho
1.0
(3,)
Tensor("strided_slice_206:0", shape=(3,), dtype=float64)
baselines
(3,)
baselines
()
107826.87763336947
eff
(2,)
eff
()
res
()
169293.14642401805


In [222]:
X = np.vstack([np.random.randn(10,2) + [2,2],
               np.random.randn(10,2) + [-2,2],
               np.random.randn(10,2) + [2,-2]])

In [223]:
X.shape

(30, 2)

In [ ]:
 #.eval(session=sess) 
            #print(sess.run(rho_term))
            print('rho')
            #print(tf.shape(f_one))
            print(rho_term)
            print(f_one.shape)
            print(f_one)
            #print(rho_term.get_shape()[1])
            #rho_term = tf.constant(np.array([1.0]))
       
            baselines = tf.tensordot(tf.transpose(f_one),self.sigma_theta,axes=[[0],[0]])
            print('baselines')
            print(baselines.get_shape())
            baselines = tf.tensordot(tf.transpose(baselines),f_two,axes=[[0],[0]])
            #print('look at shape')
            #print(baselines)
            print('baselines')
            print(baselines.get_shape())
           
            #if user_id_one==user_id_two:
            #    user_term = self.sigma_u
            #else:
            #print('user')
            #print('user_term')

            #print(self.sigma_v.shape)
            #print(rho_term.shape)
            effect_term = tf.tensordot(rho_term,self.sigma_v,axes=[[0],[0]])
                #print('effect term')
            #print(effect_term.shape)
            #print('user term')
            #print(user_term.shape)
            user_term = tf.constant(np.array([[0.0,0.0],[0.0,0.0]]))           
            effect_term = tf.add(user_term,effect_term)
            #print('eff')
            #print(effect_term.shape)
            #print('g')
            #print(g_one.shape)
            effects = tf.tensordot(tf.transpose(g_one),effect_term,axes=[[0],[1]])
            print('eff')
            print(effects.shape)
            effects = tf.tensordot(tf.transpose(effects),g_two,axes = [[0],[0]])
            print('eff')
            print(effects.shape)
            result = tf.add(baselines,effects)
            print('res')
            print(result.shape)
        

In [4]:
import pickle
with open('temp_M.pkl','rb') as f:
    M=pickle.load(f)

In [12]:
import numpy as np
np.array(M[0]).shape
import pandas as pd

In [142]:
df_rewardsa = pd.DataFrame(all_MS[0],columns=['p_1','p_2','p_3','p_4','p_5','p_6'])
path = 'M0.feather'
feather.write_dataframe(df_rewardsa , path)

In [141]:
all_MS

[[[3.182050526733636,
   0.3967589131552818,
   6.129668130181115,
   0.4108408583014009,
   6.1199090701103325,
   0.41339359993659147],
  [2.735618497832924,
   0.4546450092332019,
   5.964244510574703,
   0.42449240150178946,
   5.9130813021233655,
   0.42733408838021547],
  [2.9705708116190817,
   0.4317596370968596,
   5.71146624848382,
   0.4051573511460121,
   5.689708344993629,
   0.416677634192122],
  [3.0534396947159417,
   0.4526882426350584,
   6.228919737478516,
   0.47597293743519464,
   6.218229396267928,
   0.4402007249318048],
  [1.2716599060579667,
   0.44551685717372563,
   2.3709108762878204,
   0.42937704784495495,
   2.363934517174844,
   0.408201566164266],
  [1.7289210611382186,
   0.42393436741566953,
   2.821302626418396,
   0.4366285336886993,
   2.7778015434508023,
   0.43261882216295106],
  [0.4727951688559979,
   0.4553525937882923,
   0.5447621970644432,
   0.4225580434776346,
   0.530102034404852,
   0.44287845716929924],
  [0.8320472022148684,
   0.4064